In [ ]:
!pip install streamlit -q
!npm install localtunnel -q
!pip install sklearn.preprocessing -q
!pip install scikit-learn -q
!pip install itertools -q
!pip install xgboost -q
print("Pacotes Instalados")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.6 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧
added 22 packages in 3s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧ERROR: Could not find a version that satisfies the requirement itertools (from versions: none)
ERROR: No matching distribution found for itertools
Pacotes Instalados


In [ ]:
%%writefile Soccer_Players_Statistics_PCA.py

import streamlit as st
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, f1_score
from xgboost import XGBClassifier
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations

# Título da aplicação
st.set_page_config(page_title="Análise de Componentes Principais - Jogadores de Futebol", layout="wide")
st.title('Análise de Componentes Principais - Jogadores de Futebol')

try:
  with st.sidebar:
    uploaded_file = st.file_uploader("Faça o upload do arquivo CSV", type=["csv"])
    n_components = st.slider('Número de componentes:', 1, 5, 2)

  if uploaded_file is not None:
    df = pd.read_csv(uploaded_file)
    numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
    selected_cols = st.sidebar.multiselect('Selecione as colunas numéricas para a análise:', numeric_cols, default=numeric_cols[:5])
    process_button = st.sidebar.button("Processar")

    if process_button:
      if not selected_cols:
        st.warning('Selecione ao menos uma coluna para prosseguir.')
      else:
        st.markdown("---")
        df_selected = df[selected_cols]
        with st.expander('O que é análise de componentes principais(PCA)?'):
          st.write("""
          A análise de componentes principais (PCA) reduz o número de dimensões em grandes conjuntos de dados para os componentes principais que retêm a maior parte das informações originais. Ela faz isso transformando variáveis potencialmente correlacionadas em um conjunto menor de variáveis, chamadas componentes principais. \n""")
        st.markdown("---")

        with st.expander('Quando Utilizar a Técnica de PCA (Análise de Componentes Principais)?'):
          st.write("""A Análise de Componentes Principais (PCA) é uma técnica de redução de dimensionalidade amplamente utilizada em machine learning e estatística. Ela é especialmente útil nas seguintes situações:
          * Redução de Dimensionalidade
          * Visualização de Dados
          * Remoção de Correlação entre Variáveis
          * Pré-processamento para Outros Algoritmos
          * Tratamento de Multicolinearidade""")
        st.markdown("---")

        with st.expander('Quando não utilizar o PCA?'):
          st.write("""* Se a interpretabilidade das variáveis originais é essencial.
          * Se os dados já possuem baixa dimensionalidade.
          * Se as variáveis possuem escalas muito diferentes (nesse caso, padronize os dados antes).
          O PCA é uma ferramenta poderosa, mas seu uso deve ser justificado pelo problema e pela natureza dos dados.""")
        st.markdown("---")
        with st.expander('Visualização dos dados selecionados'):
          st.write(df_selected.head())
        st.markdown("---")

        # Lidar com valores ausentes (NaN) usando a mediana
        imputer = SimpleImputer(strategy='median')
        df_imputed = pd.DataFrame(imputer.fit_transform(df_selected), columns=df_selected.columns)

        # Padronizar os dados
        x = StandardScaler().fit_transform(df_imputed)

        # PCA
        pca = PCA(n_components=n_components)
        principalComponents = pca.fit_transform(x)
        principalDf = pd.DataFrame(data = principalComponents, columns = [f'Componente Principal {i+1}' for i in range(n_components)])

        with st.expander('Gráfico de Correlação'):
          correlation_matrix = df_selected.corr()
          fig_corr = px.imshow(correlation_matrix, text_auto=True, title='Matriz de Correlação')
          st.plotly_chart(fig_corr)
        st.markdown("---")

        # Gráfico de variância explicada
        with st.expander('Gráfico de variância explicada'):
          explained_variance_ratio = pca.explained_variance_ratio_
          cumulative_variance = np.cumsum(explained_variance_ratio)
          fig_variance = px.bar(x=[f'Componente {i+1}' for i in range(n_components)], y=explained_variance_ratio, title='Variância Explicada por Componente', labels={'x':'Componente',"y":'Variância Explicada'})
          st.plotly_chart(fig_variance)
        st.markdown("---")

        # Tabela de variância acumulada
        with st.expander('Tabela de variância acumulada'):
          df_variance = pd.DataFrame({'Componente': [f'Componente {i+1}' for i in range(n_components)], 'Variância': explained_variance_ratio})
          df_variance['Variância Acumulada'] = cumulative_variance
          st.dataframe(df_variance)
        st.markdown("---")

        #Tabela com as contribuições de cada variável por componente
        with st.expander("Tabela das contribuições de cada componente"):
          loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
          loadings_df = pd.DataFrame(loadings, columns=[f"PC{i+1}" for i in range(n_components)], index=selected_cols)
          st.dataframe(loadings_df)
        st.markdown("---")

        # Mostrar as primeiras linhas do DataFrame com os componentes
        with st.expander('Primeiras linhas do DataFrame com os componentes'):
          st.write(principalDf.head())
        st.markdown("---")
        with st.expander('Comparação dos Modelos'):
          if 'Rating' in df.columns:
            # Prepare data for classification
            df_pca_rating = pd.concat([principalDf, df['Rating']], axis=1)
            X = df_pca_rating.drop('Rating', axis=1)
            y = df_pca_rating['Rating']

            # Convert Rating to categorical (assuming it needs to be for logistic regression)
            y = pd.cut(y, bins=3, labels=[0,1,2], include_lowest=True, duplicates='drop')
            # Split data
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            # Logistic Regression
            logreg = LogisticRegression()
            logreg.fit(X_train, y_train)
            y_pred_logreg = logreg.predict(X_test)

            # XGBoost
            xgb = XGBClassifier()
            xgb.fit(X_train, y_train)
            y_pred_xgb = xgb.predict(X_test)

            # Evaluate models
            st.subheader("Avaliação dos modelos")
            models = {
                "Regressão Logística": y_pred_logreg,
                "XGBoost": y_pred_xgb
            }
            for model_name, y_pred in models.items():
              accuracy = accuracy_score(y_test, y_pred)
              recall = recall_score(y_test, y_pred,average='macro')
              f1 = f1_score(y_test, y_pred,average='macro')
              st.write(f"{model_name}:")
              st.write(f"- Precisão: {accuracy:.4f}")
              st.write(f"- Recall: {recall:.4f}")
              st.write(f"- F1-score: {f1:.4f}")
              st.markdown("---")
          else:
            st.warning("Coluna 'Rating' não encontrada no dataset. Não foi possível realizar a análise.")
            st.markdown("---")
    else:
      st.write("Aguardando o clique em 'Processar'...")
  else:
    st.warning("Por favor, faça o upload de um arquivo CSV.")
except:
      st.warning("Por favor, faça o upload de um arquivo CSV.")


Overwriting Soccer_Players_Statistics_PCA.py


In [ ]:
!curl ipv4.icanhazip.com

34.150.167.133


In [ ]:
!streamlit run Soccer_Players_Statistics_PCA.py & npx localtunnel --port 8501



⠙
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.150.167.133:8501

your url is: https://hungry-lies-call.loca.lt
